In [1]:
import tensorflow as tf
from tensorflow import keras

# SCA koriscenjem deep learninga

U radu je do sada bilo prikazano nekoliko nacina izvlacenja tajni iz hardvera. State of the art pristup su template napadi koji su se pokazali kao standard u ovoj oblasti istrazivanja sigurnosti sistema. Kod template napada je potrebno ozbiljno preprocesiranje ulaznih podataka, potrebno je dosta iskustva. DPA i CPA su dovoljno jednostavni za koriscenje i implementaciju, ali je i za njih potreban relativno veliki broj trace-va i ne pokazuju se toliko dobro sa sumovitim podacima. Deep learning nastoji da eliminise neke od navedenih problema. 

# Arhitekture neuronskih mreza koje ce biti koriscene

U radu koji opisuje ASCAD bazu podataka date su i neke smernice oko odabira odgovarajucih modela i izbora hiperparametara. Preprocesiranje podataka i redukcija dimenzionalnosti nije potrebna, jer se ona obavi u prvim slojevima mreze.

### MLP (multi-layered perceptron)

Obicna potpuno povezana neuronska mreza sa softmax aktivacijom - normalizuje izlaze

input sloj : 80.000 tacaka traceva
output sloj : 256 klasa za 256 mogucih vrednosti jednog bajta kljuca


### ResNet

### CNN (Convolutional Neural Network)

In [2]:
EXECUTE_IN_COLAB = True

if EXECUTE_IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import os

def load_shards(path, num_shards=256):
  shard_array = []
  for shard_idx, shard_name in enumerate(os.listdir(path)):
    if shard_idx == num_shards:
      break
    shard_array.append(np.load(path + '/' + shard_name))

  return shard_array

In [4]:
training_data_path = '/content/drive/MyDrive/datasets/tinyaes/train'
testing_data_path = '/content/drive/MyDrive/datasets/tinyaes/test'

training_data = load_shards(training_data_path, num_shards=50)
# test_data = load_shards(testing_data_path)

In [5]:
print(training_data[0]['traces'][:100, :40000,].shape)
print(training_data[0]['sub_bytes_out'][:100, :40000,].shape)

(100, 40000, 1)
(16, 256)


In [6]:
training_data[0]['traces'].shape

(256, 80000, 1)

In [7]:
from tqdm.notebook import tqdm

X_train = []
y_train = []

for training_shard in tqdm(training_data, desc='Loading training shards', position=0, leave=False):
  for sample, label in zip(training_shard['traces'], training_shard['sub_bytes_out'].transpose()):
    X_train.append(sample)
    y_train.append(label)

Loading training shards:   0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
X_train = np.array(X_train)

In [9]:
y_train = np.array(y_train)

In [10]:
import keras
from keras.utils import np_utils

print(y_train[0].shape)
# sample idx _
#             |
#             v  v------ attack byte
print(y_train[0][1])
keras.utils.np_utils.to_categorical(y_train[0][1], dtype='uint8', num_classes=256)

(16,)
15


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [11]:
import keras
from keras.utils import np_utils

A = [1,3,3]

keras.utils.np_utils.to_categorical(A, dtype="uint8")

array([[0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1]], dtype=uint8)

In [12]:
training_data[0]['sub_bytes_out']

array([[ 71,  13,  22, ..., 250, 180,  41],
       [ 15, 139, 171, ..., 154, 213, 223],
       [160, 123,  27, ..., 171, 105,  48],
       ...,
       [213, 189, 237, ..., 182, 205,  56],
       [199, 116, 223, ..., 109, 153, 124],
       [ 75,  19,  88, ..., 218, 196, 199]], dtype=uint8)

In [13]:
# ATTACK_POINT = 'sub_bytes_out'

# training_X = training_data['traces']
# training_y = training_data[ATTACK_POINT]

TypeError: ignored

In [17]:
MLP_test = keras.models.Sequential()

input_size = X_train.shape[0]

# Input layer
MLP_test.add(
    keras.layers.Dense(
        name='input_layer',
        units=10000,
        input_dim=X_train.shape[0],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'
    )
)

# Hidden layer 1
MLP_test.add(
    keras.layers.Dense(
        name='layer_1',
        units=5000,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'
    )
)

# Hidden layer 2
MLP_test.add(
    keras.layers.Dense(
        name='layer_2',
        units=2500,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'
    )
)

# Hidden layer 3
MLP_test.add(
    keras.layers.Dense(
        name='layer_3',
        units=1250,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'
    )
)

# Hidden layer 4
MLP_test.add(
    keras.layers.Dense(
        name='layer_4',
        units=512,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'
    )
)

# Output layer
MLP_test.add(
    keras.layers.Dense(
        name='output_layer',
        units=256,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'
    )
)

MLP_test_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
MLP_test.compile(loss='categorical_crossentropy', optimizer=MLP_test_optimizer)

In [18]:
MLP_test.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 10000)             128010000 
                                                                 
 layer_1 (Dense)             (None, 5000)              50005000  
                                                                 
 layer_2 (Dense)             (None, 2500)              12502500  
                                                                 
 layer_3 (Dense)             (None, 1250)              3126250   
                                                                 
 layer_4 (Dense)             (None, 512)               640512    
                                                                 
 output_layer (Dense)        (None, 256)               131328    
                                                                 
Total params: 194,415,590
Trainable params: 194,415,59

In [ ]:
labels = keras.utils.np_utils.to_categorical(y_train)
X_train_ = X_train[:,:,0]

tf.convert_to_tensor(labels, dtype='uint8')
tf.convert_to_tensor(X_train_, dtype='float32')

tf.concat(labels, axis=0)
tf.concat(X_train, axis=0)

history = MLP_test.fit(
    X_train_,
    labels,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1
)